## Walk through of a sample PCL code:


### Lesson 1 Create and SVM with iSCSI Example

[create_san.py](../../create_san.py)

#### The Header Comment Section

This section usually includes details such as 
- the file name, 
- author, 
- date, 
- purpose of the program, and any other relevant information. 

It’s a good practice to include such comments for better readability and maintainability of the code 

In [ ]:
#! /usr/local/bin/python3

"""
Purpose: Script to create sn SVM by using the netapp_ontap library.
         It will create a SVM, a volume, two data interfaces and enable iscsi.
Author: Vish Hulikal
Usage: python3.11 create_san.py [-h] -c CLUSTER -a AGGR_NAME, -n NODE_NAME, -vs VSERVER_NAME, -v VOLUME_NAME -ip DATA_LIF,
                 -lif INTERFACE_NAME,   -nm NET_MASK, [-u API_USER] [-p API_PASS]
"""

#### The Input Section:

Since we are using a Jupyter Notebook to run the code and not running the code from the command line, we need this section to simulate entering the arguments from the command line.

When you run the notebook, for example by clicking on the `Run All` button, a dialog prompt will appear at the top of the window. You will then need to enter the command line arguments as described in the usage section above.

If you do not provide the user, `admin`is used. If you do not provide the password, you will be prompted for the password.

Suggested command arguments:

In [ ]:
import sys

# Prompt the user to enter command line arguments
args = input("Please enter command line arguments: ")

# Split the entered string into a list of arguments
args = args.split()

# Assign the list of arguments to sys.argv
sys.argv = ['ipykernel_launcher.py'] + args

#### Import Section

Here we will import the following modules:
  - [`argparse`](https://pypi.org/project/argparse/) : This is a popular python module. The argparse module makes it easy to write user friendly command line interfaces. 
  The program defines what arguments it requires, and argparse will figure out how to parse those out of sys.argv. The argparse module also automatically generates help and usage messages and issues errors when users give the program invalid arguments.
  - [`getpass`](https://docs.python.org/3/library/getpass.html) : Used to Prompt the user for a password without echoing
  - [`logging`](https://pypi.org/project/logging/) : This module is intended to provide a standard error logging mechanism in Python as per PEP 282.
  - [`netapp_ontap.config`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/config.html) : This module contains the global configuration options and related functions for the library.
  - [`netapp_ontap.host_connection`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/host_connection.html) : This module defines a host connection object which is used to communicate with the API host
  - [`netapp_ontap.error`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/error.html) : This module defines the custom exception type. All exceptions raised by the library descend from this type
  - [`netapp_ontap.resources.svm`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/resources/svm.html) : Managing SVMs
  - [`netapp_ontap.resources.volume`](https://library.netapp.com/ecmdocs/ECMLP2885777/html/resources/volume.html) : Manage Volumes 
  - [`netapp_ontap.resources.ip_interface`](https://library.netapp.com/ecmdocs/ECMLP2885777/html/resources/ip_interface.html) : Manage IP LIF
  - [`netapp_ontap.resources.iscsi_service`](https://library.netapp.com/ecmdocs/ECMLP2885777/html/resources/iscsi_service.html) : Manage iSCSI Service


In [ ]:

import argparse
from getpass import getpass
import logging

from netapp_ontap import config, utils, HostConnection, NetAppRestError
from netapp_ontap.resources import Svm, Volume, IpInterface, IscsiService


#### Function Definitions

These functions are defined:
  
  - `create_svm`
  - `create_iscsi_service`  
  - `create_volume`  
  - `create_data_interface`  

  

In [ ]:
def create_svm(vserver_name: str, aggr_name: str) -> None:
    """Create an SVM on the specified aggregate"""

    svm = Svm.from_dict({
    'name': vserver_name,
    'aggregates': [{'name': aggr_name}],
#    'iscsi': {'enabled': "true"}
    })

    try:
        svm.post()
        print("SVM %s created successfully" % svm.name)
    except NetAppRestError as err:
        print("Error: SVM was not created: %s" % err)
    return


In [ ]:

def create_iscsi_service(vserver_name: str) -> None:
    """Enable iSCSI service for an SVM"""

    data = {
        'svm': {'name': vserver_name},
    }

    iScsiService = IscsiService(**data)

    try:
        iScsiService.post()
        print("iSCSI Service %s created successfully" % vserver_name)
    except NetAppRestError as err:
        print("Error: iScsi Service was not created: %s" % err)
    return


In [ ]:

def create_volume(volume_name: str, vserver_name: str, aggr_name: str, volume_size: int) -> None:
    """Creates a new volume in a SVM"""

    data = {
        'name': volume_name,
        'svm': {'name': vserver_name},
        'aggregates': [{'name': aggr_name }],
        'size': volume_size,
        'space_guarantee': 'volume'
    }

    volume = Volume(**data)

    try:
        volume.post()
        print("Volume %s created successfully" % volume.name)
    except NetAppRestError as err:
        print("Error: Volume was not created: %s" % err)
    return



In [ ]:


def create_data_interface(vserver_name: str, interface_name: str, node_name: str, ip_address: str, ip_netmask: str) -> None:
    """Creates an SVM-scoped IP Interface"""

    data = {
        'name': interface_name,
        'ip': {'address': ip_address, 'netmask': ip_netmask},
        'enabled': True,
        'scope': 'svm',
        'svm': {'name': vserver_name},
        'port': {'name': 'e0d', 'node': node_name},
        'location': {
           'auto_revert': True,
           'broadcast_domain': {'name': 'Default'},
        }
    }

    ip_interface = IpInterface(**data)

    try:
        ip_interface.post()
        print("Ip Interface %s created successfully" % ip_interface.ip.address)
    except NetAppRestError as err:
        print("Error: IP Interface was not created: %s" % err)
    return



#### Arguments Parser

We define which arguments need to be passed to the script and argparse does the rest...

In [ ]:
def parse_args() -> argparse.Namespace:
    """Parse the command line arguments from the user"""

    parser = argparse.ArgumentParser(
        description="This script will create a SAN configuration"
    )
    parser.add_argument(
        "-c", "--cluster", required=True, help="Cluster Name"
    )
    parser.add_argument(
        "-n", "--node_name", required=True, help="API server Node Name"
    )
    parser.add_argument(
        "-a", "--aggr_name", required=True, help="Aggregate name"
    )
    parser.add_argument(
        "-vs", "--vserver_name", required=True, help="VServer name"
    )
    parser.add_argument(
        "-v", "--volume_name", required=True, help="Volume name"
    )
    parser.add_argument(
        "-ip", "--ip_address", required=True, help="Data Interface IP Address"
    )
    parser.add_argument(
        "-nm", "--ip_netmask", required=True, help="DNS Server IP Address"
    )
    parser.add_argument(
        "-lif", "--interface_name", required=True, help="Interface name"
    )

    parser.add_argument("-u", "--api_user", default="admin", help="API Username")
    parser.add_argument("-p", "--api_pass", help="API Password")
    parsed_args = parser.parse_args()

    # collect the password without echo if not already provided
    if not parsed_args.api_pass:
        parsed_args.api_pass = getpass()

    return parsed_args


#### The Main Section

In [ ]:
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format="[%(asctime)s] [%(levelname)5s] [%(module)s:%(lineno)s] %(message)s",
    )

    args = parse_args()
    config.CONNECTION = HostConnection(
        args.cluster, username=args.api_user, password=args.api_pass, verify=False,
    )

    create_svm(args.vserver_name, args.aggr_name)
    create_iscsi_service(args.vserver_name)
    create_volume(args.volume_name, args.vserver_name, args.aggr_name, 300000000)
    create_data_interface(args.vserver_name, args.interface_name, args.node_name, args.ip_address, args.ip_netmask)


---

---

[Back to Module 6 Demo - Notebook:](./Module6_demo.ipynb)